In [1]:
import os
from pathlib import Path
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from modules.functions import bound_logistic
from modules.plotstyle import PlotStyle
import matplotlib.colors as mcolors
import plottools.colors as clrs
import cmocean.cm as cmo
import seaborn as sns
from scipy.stats import wilcoxon
from cvs_preprocessing import dataimport
import matplotlib.colors as mcolors
%matplotlib qt

# init standardized plotstyle
ps = PlotStyle()

In [2]:
# import the data
dataroot = Path("../data/cvs")
processed_dataroot = Path("../data_processed")
df = dataimport(dataroot, processed_dataroot)

# delete JONA!
df = df[df.subj != 'VP1']

def mean_sem(df):
    # means and sem for trial duration
    names = np.unique(df.subj).tolist()
    n = df.shape[1] / len(names)
    means = np.array(df.tpertrial.groupby(df.subj).mean())
    sems = np.array(df.tpertrial.groupby(df.subj).std()) / np.sqrt(n)

    # put to dataframe
    df_all = pd.DataFrame(
        list(zip(names, means, sems)),
        columns=("names", "tpertrial_means", "tpertrial_sems"),
    )

    # means and sems for no of switches
    means = np.array(df.switches.groupby(df.subj).mean())
    sems = np.array(df.switches.groupby(df.subj).std()) / np.sqrt(n)
    df_all["switches_means"] = means
    df_all["switches_sems"] = sems

    return df_all

In [3]:
# fit power law function
def func_powerlaw(x, m, c, c0):
    return c0 + x**m * c
df_all = mean_sem(df)
popt, pcov = curve_fit(func_powerlaw, df_all.switches_means, df_all.tpertrial_means, method="trf", maxfev=50000)

### Plot mean and sem for switches and handling time for every subject

In [4]:
# plot data for all subjects
colors = mcolors.TABLEAU_COLORS


def plot_errorbars(df_all, ax):
    for name, color in zip(df_all.names, colors):
        minidf = df_all[df_all.names == name]
        ax.errorbar(
            minidf.switches_means,
            minidf.tpertrial_means,
            xerr=minidf.switches_sems,
            yerr=minidf.tpertrial_sems,
            fmt="bo",
            label=name,
            color=color,
        )
fig, ax = plt.subplots(1,2, figsize=(24*ps.cm, 12*ps.cm), constrained_layout=True)
for i in range(len(ax)):
    plot_errorbars(df_all, ax[i])

x = np.linspace(df_all.switches_means.min()-0.2, df_all.switches_means.max()+2, 1000)

ax[1].plot(x, func_powerlaw(x, *popt))
ax[0].plot(x, func_powerlaw(x, *popt))
ax[1].set_ylim(0, 6)

fig.supxlabel('Number of switches')
fig.supxlabel(' of switches')
plt.show()

/tmp/ipykernel_43451/2794611695.py:8: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "bo" (-> color='b'). The keyword argument will take precedence.
  ax.errorbar(


### Plot seperated for delay conditiom

In [5]:

def plot_errorbars2(df_all, ax, color):
    for name, _ in zip(df_all.names, colors):
        minidf = df_all[df_all.names == name]
        ax.errorbar(
            minidf.switches_means,
            minidf.tpertrial_means,
            xerr=minidf.switches_sems,
            yerr=minidf.tpertrial_sems,
            fmt="bo",
            label=name,
            color=color,
        )
# sort dataset by delay condition
df_0 = df[df.delay == 0]
df_2 = df[df.delay == 1.5]

ms_0 = mean_sem(df_0)
ms_2 = mean_sem(df_2)

popt0, pcov = curve_fit(func_powerlaw, ms_0.switches_means, ms_0.tpertrial_means, method="trf", maxfev=50000)
popt2, pcov = curve_fit(func_powerlaw, ms_2.switches_means, ms_2.tpertrial_means, method="trf", maxfev=50000)

fig, ax = plt.subplots()
plot_errorbars2(ms_0, ax, color='red')
plot_errorbars2(ms_2, ax, color='blue')

ax.plot(x, func_powerlaw(x, *popt0))
ax.plot(x, func_powerlaw(x, *popt2))

plt.show()

/tmp/ipykernel_43451/1876838165.py:4: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "bo" (-> color='b'). The keyword argument will take precedence.
  ax.errorbar(


### Plot seperated for cycles per degree condition

In [6]:
# sort dataset by delay condition
df_0 = df[df.cpd == 2]
df_2 = df[df.cpd == 8]

ms_0 = mean_sem(df_0)
ms_2 = mean_sem(df_2)

popt0, pcov = curve_fit(func_powerlaw, ms_0.switches_means, ms_0.tpertrial_means, method="trf", maxfev=50000)
popt2, pcov = curve_fit(func_powerlaw, ms_2.switches_means, ms_2.tpertrial_means, method="trf", maxfev=50000)

fig, ax = plt.subplots()
plot_errorbars(ms_0, ax)
plot_errorbars(ms_2, ax)

ax.plot(x, func_powerlaw(x, *popt0))
ax.plot(x, func_powerlaw(x, *popt2))

plt.show()

/tmp/ipykernel_43451/2794611695.py:8: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "bo" (-> color='b'). The keyword argument will take precedence.
  ax.errorbar(


### Compute selectivity index

In [7]:
# compute mean for every subject

def getindex(df):

    mean = []
    sem = []
    names = []
    for name in np.unique(df.subj):
        print(np.min(df.strat_idx[df.subj==name]))
        print(np.max(df.strat_idx[df.subj==name]))
        mean.append(np.mean(df.strat_idx[df.subj==name]))
        sem.append(np.std(df.strat_idx[df.subj==name]))
        names.append(name)

    return mean, sem, names

mean, sem, names = getindex(df)

0.0039335210940703
0.0386629254512326
0.0034400523005874
0.0284508141780779
0.0008928828644568
0.0481080631378082
0.0011384791820448
0.0125131740650375
0.0036470304202563
0.0225088771717522
0.0049845793151208
0.025376040149706
0.0004211719940796
0.0173417920975854
0.0254548236055034
1.0004211719940796


In [8]:
fig, ax = plt.subplots()

for i, name in enumerate(names):
    plt.errorbar(mean[i], i, xerr=sem[i], fmt='bo')

ax.set_xlim(0,1)

plt.show()

In [9]:
# repeat for two categories
fig, ax = plt.subplots()

mean, sem, names = getindex(df[df.delay==0])

for i, name in enumerate(names):
    plt.errorbar(mean[i], i, xerr=sem[i], fmt='bo')


mean, sem, names = getindex(df[df.delay==1.5])

for i, name in enumerate(names):
    plt.errorbar(mean[i], i+0.2, xerr=sem[i], fmt='ro')

ax.set_xlim(0,1)

plt.show()

0.0039335210940703
0.035960763879998
0.0034400523005874
0.0186969784999133
0.0008928828644568
0.0216219441226473
0.0011384791820448
0.0049178268108839
0.0036668705356035
0.0225088771717522
0.0049845793151208
0.025376040149706
0.0010270011112126
0.0072598153120584
0.0254548236055034
0.6685453096347596
0.0075518935699361
0.0386629254512326
0.0042575911064242
0.0284508141780779
0.0039325005399624
0.0481080631378082
0.0018820438278091
0.0125131740650375
0.0036470304202563
0.0172589030250775
0.0071291260113617
0.0219216630195276
0.0004211719940796
0.0173417920975854
0.048316339963909
1.0004211719940796
